In [43]:
# imports
import numpy as np
from collections import defaultdict

In [44]:
def read_dataset(path, bool=True):
    with open(path, encoding="utf-8") as filepath:
        data = []
        line_lst = []
        for line in filepath:
            if line != "\n":
                if bool:
                    tokens = line.strip().split()
                    line_lst.append((" ".join(tokens[: -1]), tokens[-1]))
                else:
                    line_lst.append((line.strip(), ))
            else:
                data.append(line_lst)
                line_lst = []
        return data

In [45]:
def write_dataset(path, data):
    with open(path, "w", encoding="utf-8") as filepath:
        for data_str in data:
            for line in data_str:
                filepath.write(f"{line[0]} {line[1]}\n")
            filepath.write("\n")

# Part 1
Write a function that estimates the emission parameters from the training set using MLE (maximum
likelihood estimation):

`e(x|y) = Count(y → x) / Count(y)`

In [46]:
def get_estimate_with_mle(dataset):
    words = set([x for line in dataset for (x, y) in line])
    y_count = defaultdict(int)
    xy_count = defaultdict(lambda  : y_count)

    for line in dataset:
        for word, label in line:
            y_count[label] += 1
            xy_count[label][word] += 1

    def est(x, y):
        return xy_count[y][x] / y_count[y]
        
    return est

One problem with estimating the emission parameters is that some words that appear in the test set
do not appear in the training set. One simple idea to handle this issue is as follows. We introduce
a special word token `#UNK#`, and make the following modifications to the computation of emission
probabilities:

```
e(x|y) = Count(y→x) / Count(y)+k, 
If the word token x appears in the training set

e(x|y) = k / Count(y)+k,
If word token x is the special token #UNK#
```

(This basically says we assume from any label y there is a certain chance of generating `#UNK#` as
a rare event, and empirically we assume we have observed that there are k occurrences of such an
event.)

During the testing phase, if the word does not appear in the training set, we replace that word with
`#UNK#`.

Set k to 1, implement this fix into your function for computing the emission parameters.

In [47]:
def get_estimate(dataset, k=1):
    words = set([x for line in dataset for (x, y) in line])
    y_count = defaultdict(int)
    xy_count = defaultdict(lambda  : y_count)
    
    for line in dataset:
        for word, label in line:
            y_count[label] += 1
            xy_count[label][word] += 1
            
    def e_para(x, y):
        if x in words:
            return xy_count[y][x]/(y_count[y] + k)
        else:
            return k/(y_count[y] + k)

    return e_para

For all the datasets `RU`, `ES`, learn these parameters with `train`, and evaluate your system on the
development set `dev.in` for each of the dataset. Write your output to `dev.p1.out` for the two
datasets respectively.

Compare your outputs and the gold-standard outputs in `dev.out` and report
the precision, recall and F scores of such a baseline system for each dataset.

The precision score is defined as follows:

`Precision = (Total number f correctly predicted entities) / (Total number of predicted entities)`

The recall score is defined as follows:

`Recall = (Total number of correctly predicted entities) / (Total number of gold entities)`

where a gold entity is a true entity that is annotated in the reference output file, and a predicted entity
is regarded as correct if and only if it matches exactly the gold entity (i.e., both their boundaries and
sentiment are exactly the same).

Finally the F score is defined as follows:

`F = 2 / [(1/Precision) + (1/Recall)]`

*Note: in some cases, you might have an output sequence that consists of a transition from `O` to
`I-negative` (rather than `B-negative`). For example, “`O I-negative I-negative O`”.
In this case, the second and third words should be regarded as one entity with negative sentiment.*

You can use the evaluation script shared with you to calculate such scores. However it is strongly
encouraged that you understand how the scores are calculated.

In [48]:
def labelling(dataset, trainset):
    ys = set([y for line in trainset for (x,y) in line])
    estimate = get_estimate(trainset)
    func_output = []
    for line in dataset:
        labels = []
        for word, in line:
            max_p = 0
            label = None
            for y in ys:
                p = estimate(word, y)
                if p > max_p:
                    max_p = p
                    label = y
            labels.append((word, label))
        func_output.append(labels)
    return func_output

## ES

In [49]:
es_trainSet = read_dataset("./ES/ES/train")
es_dev = read_dataset("./ES/ES/dev.in", False)
es_dev_label = labelling(es_dev, es_trainSet)
write_dataset("./ES/ES/dev.p2.out", es_dev_label)

*insert some report stuff*

## RU

In [50]:
ru_trainSet = read_dataset("./RU/RU/train")
ru_dev = read_dataset("./RU/RU/dev.in", False)
ru_dev_label = labelling(ru_dev, ru_trainSet)
write_dataset("./RU/RU/dev.p2.out", ru_dev_label)

*insert some report stuff*

# Part 2
Write a function that estimates the transition parameters from the training set using MLE (maximum
likelihood estimation):

`q(yi|yi−1) = Count(yi−1, yi) / Count(yi−1)`

Please make sure the following special cases are also considered: q(STOP|yn) and q(y1|START).

In [51]:
def get_q_para(dataset, k=1):
    states = defaultdict(lambda  : defaultdict(int))
    y_count = defaultdict(int)
    y_count['START'] = len(dataset)
    y_count['STOP'] = len(dataset)
    for sent in dataset:
        n = len(sent)
        for i in range(n):
            y_current = sent[i][1]
            y_count[y_current] += 1
            if i == 0:
                states[y_current]['START'] += 1
            elif i == n-1:
                states['STOP'][y_current] += 1
                states[y_current][sent[i-1][1]] += 1
            else:
                states[y_current][sent[i-1][1]] += 1
    def q(yi, yi_1):
        if states[yi][yi_1] == 0:
            return k/(y_count[yi_1] + k) # smoothing
        return states[yi][yi_1]/((y_count)[yi_1] + k)
    return q

Use the estimated transition and emission parameters, implement the Viterbi algorithm to compute
the following (for a sentence with n words):

```
y1,...,yn = arg max p(x1,...,xn, y1,...,yn)
           y1,...,yn
```

For all datasets, learn the model parameters with `train`. Run the Viterbi algorithm on the development set `dev.in` using the learned models, write your output to `dev.p2.out` for the two datasets respectively. Report the precision, recall and F scores of all systems.

*Note: in case you encounter potential numerical underflow issue, think of a way to address such an
issue in your implementation.*

# Part 3
Use the estimated transition and emission parameters, implement an algorithm to find the 5-th best
output sequences (if there are multiple sequences that are 5-th best, output any of them). Clearly
describe the steps of your algorithm in your report.

Run the algorithm on the development sets `RU/dev.in` and `ES/dev.in`. Write the outputs to
`RU/dev.p3.out` and `ES/dev.p3.out`. Report the precision, recall and F scores for the outputs
for both languages.

*Hint: find the top-5 best sequences using dynamic programming by modifying the original Viterbi
algorithm*

# Part 4
Now, based on the training and development set, think of a better design for developing an improved sentiment analysis system for tweets using any model you like. Please explain clearly the model/method that you used for designing the new system. We will check your code and may call you for an interview if we have questions about your code. Please run your system on the development set `RU/dev.in` and `ES/dev.in`.

Write your outputs to `RU/dev.p4.out` and `ES/dev.p4.out`.

Report the precision, recall and F scores of your new systems for these two languages.

We will evaluate your system’s performance on two held out test sets `RU/test.in` and `ES/test.in`. The test sets will only be released on 48 hours before the deadline. Use your new system to generate the outputs.

Write your outputs to `RU/test.p4.out` and `ES/test.p4.out`.

The system that achieves the overall highest F score on the test sets will be announced as the winner.

*Hints: can we handle the new words in a better way? Are there better ways to model the transition and emission probabilities? Or can we use a discriminative approach instead of the generative approach? Perhaps using Perceptron? Any other creative ideas? Note that you are allowed to look into the scientific literature for ideas.*